In [16]:
# Simple CNN
import torch
from torch import nn
import torch.nn.functional as F
class CNN(nn.Module):
    def __init__(self,input_size=1,num_classes=10):
        super(CNN,self).__init__()
        self.conv1=nn.Conv2d(in_channels=1,out_channels=16,kernel_size=(3,3),stride=(1,1),padding=(1,1))
        self.conv2=nn.Conv2d(in_channels=16,out_channels=32,kernel_size=(3,3),stride=(1,1),padding=(1,1))
        self.pool=nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))
        self.fc1=nn.Linear(in_features=32*7*7,out_features=num_classes)

    def forward(self,x):
        x=F.relu(self.conv1(x))
        x=self.pool(x)
        x=F.relu(self.conv2(x))
        x=self.pool(x)
        x=x.reshape(x.shape[0],-1)

        x=self.fc1(x)

        return x


In [17]:
model=CNN()

x=torch.randn(64,1,28,28)

In [18]:
device="cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [19]:
in_channel=1
num_classes=10
learrning_rate=0.001
batch_size=64
num_epochs=10

In [20]:
# Load data
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.optim import Adam

train_dataset=datasets.MNIST(root="dataset/",train=True,transform=transforms.ToTensor(),download=True)
train_loader=DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)
test_dataset=datasets.MNIST(root="datasets/",train=False,transform=transforms.ToTensor(),download=True)
test_dataloader=DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)


Files already downloaded
Files already downloaded


    .. math::
        \ell(x, y) = L = \{l_1,\dots,l_N\}^\top, \quad
        l_n = - w_{y_n} x_{n,y_n}, \quad
        w_{c} = \text{weight}[c] \cdot \mathbb{1}\{c \not= \text{ignore\_index}\},

    where :math:`x` is the input, :math:`y` is the target, :math:`w` is the weight, and
    :math:`N` is the batch size. If :attr:`reduction` is not ``'none'``
    (default ``'mean'``), then

In [28]:
def check_accuracy(loader,model):
    if loader.dataset.train:
        print("checking accuracy on training data")
    else:
        print("Checking accuracy on test data")
    num_correct=0
    num_samples=0
    model.eval()

    with torch.no_grad():
        num_correct=0
        num_samples=0
        model.eval()

        with torch.no_grad():
            for x,y in loader:
                x=x.to(device)
                y=y.to(device)
                # x=x.reshape(x.shape[0],-1)
                scores=model(x)
                _,predictions=scores.max(1)
                num_correct+=(predictions==y).sum()
                num_samples+=predictions.shape[0]
            acc= float(num_correct)/float(num_samples)*100
            print(f"Got {num_correct} / {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f}")
            model.train()
            return acc

In [29]:
from torch import optim
device="cuda"
model=CNN().to(device)
criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=learrning_rate)

In [30]:
# Train Network
for epoch in range(num_epochs):
    for batch_idx,(data,targets) in enumerate(train_loader):
        data=data.to(device)
        targets=targets.to(device)
        # data=data.reshape(data.shape[0],-1)
        scores=model(data)
        loss=criterion(scores,targets)
        # bacckward
        optimizer.zero_grad()
        loss.backward()
        # gradient descent
        optimizer.step()
    check_accuracy(train_loader,model)
    check_accuracy(test_dataloader,model)


checking accuracy on training data
Got 58691 / 60000 with accuracy 97.82
Checking accuracy on test data
Got 9782 / 10000 with accuracy 97.82
checking accuracy on training data
Got 59117 / 60000 with accuracy 98.53
Checking accuracy on test data
Got 9861 / 10000 with accuracy 98.61
checking accuracy on training data
Got 59218 / 60000 with accuracy 98.70
Checking accuracy on test data
Got 9848 / 10000 with accuracy 98.48
checking accuracy on training data
Got 59260 / 60000 with accuracy 98.77
Checking accuracy on test data
Got 9836 / 10000 with accuracy 98.36
checking accuracy on training data
Got 59403 / 60000 with accuracy 99.00
Checking accuracy on test data
Got 9861 / 10000 with accuracy 98.61
checking accuracy on training data
Got 59455 / 60000 with accuracy 99.09
Checking accuracy on test data
Got 9866 / 10000 with accuracy 98.66
checking accuracy on training data
Got 59523 / 60000 with accuracy 99.20
Checking accuracy on test data
Got 9873 / 10000 with accuracy 98.73
checking accu